In [2]:
from instruments import Voltmeter,Sourcemeter,VSourcemeter,MercuryiTC,MercuryiPS
from instrument_group import InstrumentGroup
import pyvisa
from time import sleep

### See available instruments

In [3]:
rm = pyvisa.ResourceManager()
inst = rm.list_resources()
print(inst)

### Initialize instruments and create the instrument group

In [ ]:

# Initialise all instruments and create the instrument group

voltmeters = [Voltmeter('GPIB0::6::INSTR'),Voltmeter('GPIB0::12::INSTR'),Voltmeter('GPIB0::22::INSTR'),Voltmeter('GPIB0::25::INSTR')]
sourcemeters = [Sourcemeter('GPIB0::5::INSTR'),Sourcemeter('GPIB0::11::INSTR')]
Vsourcemeters = [VSourcemeter('GPIB0::23::INSTR'),VSourcemeter('GPIB0::24::INSTR')]
iTC = MercuryiTC('ASRL7::INSTR')
iPS = MercuryiPS('ASRL8::INSTR')
IG = InstrumentGroup(voltmeters, sourcemeters, Vsourcemeters, iTC, iPS)

### Run your script

In [ ]:
### Valid commands are:
# sleep(seconds)
# IG.ramp_VTI(filename,temp,rate,threshold=0.05,timeout_hours=12)
# IG.ramp_probe(filename,temp,rate,threshold=0.05,timeout_hours=12)
# IG.ramp_probe_and_VTI(filename,temp,rate,threshold=0.05,timeout_hours=12)